In [57]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
from fastlane_bot import Bot, Config
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.helpers.poolandtokens import PoolAndTokens
from fastlane_bot.helpers import TradeInstruction, TxReceiptHandler, TxRouteHandler, TxSubmitHandler, TxHelpers, TxHelper
from fastlane_bot.events.manager import Manager
from fastlane_bot.events.interface import QueryInterface
import pytest
import math
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *

plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v2.10.1 (07/May/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.1 (2023-07-03)
UniswapV3 v0.0.1 (2023-07-03)
SushiswapV2 v0.0.1 (2023-07-03)
CarbonV1 v0.0.1 (2023-07-03)
BancorV3 v0.0.1 (2023-07-03)
Version = 3-b2.2 [requirements >= 3.0 is met]


/var/folders/rt/qnj8r6yd6131ccxkw_k9d9gc0000gn/T/ipykernel_59284/11046064.py:25: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


In [58]:
C = cfg = Config.new(config=Config.CONFIG_MAINNET)
assert (C.NETWORK == C.NETWORK_MAINNET)
assert (C.PROVIDER == C.PROVIDER_ALCHEMY)
setup_bot = CarbonBot(ConfigObj=C)

In [59]:
import json

pools = None
with open('latest_pool_data.json') as f:
    pools = json.load(f)
pools = [pool for pool in pools]

In [60]:
pools[0]
static_pools = pools

In [61]:
print(pools[0])
print(static_pools[0])

{'cid': '0x06b295766e941eb449be6a47d11466a50debbacc20638269523dac448fd210d4', 'last_updated': nan, 'last_updated_block': 17656341, 'descr': 'uniswap_v3 WETH-6Cc2/LHINU-038d 3000', 'pair_name': 'WETH-6Cc2/LHINU-038d', 'exchange_name': 'uniswap_v3', 'fee': 3000, 'fee_float': 0.003, 'address': '0xc5BBA4469eD5B55eE21228C9E7261816081C4Bfb', 'anchor': nan, 'tkn0_address': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'tkn1_address': '0xCeDefE438860D2789dA6419b3a19cEcE2A41038d', 'tkn0_key': 'WETH-6Cc2', 'tkn1_key': 'LHINU-038d', 'tkn0_decimals': 18.0, 'tkn1_decimals': 18.0, 'exchange_id': 4.0, 'tkn0_symbol': 'WETH', 'tkn1_symbol': 'LHINU', 'timestamp': nan, 'tkn0_balance': nan, 'tkn1_balance': nan, 'liquidity': 0, 'sqrt_price_q96': 377342446735349330393098223164703, 'tick': 169379, 'tick_spacing': 60, 'exchange': 'uniswap_v3'}
{'cid': '0x06b295766e941eb449be6a47d11466a50debbacc20638269523dac448fd210d4', 'last_updated': nan, 'last_updated_block': 17656341, 'descr': 'uniswap_v3 WETH-6Cc2/LHINU-

In [62]:
state = pools
exchanges = list({ex['exchange_name'] for ex in state})
db = QueryInterface(state=state, ConfigObj=C, exchanges=exchanges)
setup_bot.db = db

In [63]:
print(state[0])

{'cid': '0x06b295766e941eb449be6a47d11466a50debbacc20638269523dac448fd210d4', 'last_updated': nan, 'last_updated_block': 17656341, 'descr': 'uniswap_v3 WETH-6Cc2/LHINU-038d 3000', 'pair_name': 'WETH-6Cc2/LHINU-038d', 'exchange_name': 'uniswap_v3', 'fee': 3000, 'fee_float': 0.003, 'address': '0xc5BBA4469eD5B55eE21228C9E7261816081C4Bfb', 'anchor': nan, 'tkn0_address': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'tkn1_address': '0xCeDefE438860D2789dA6419b3a19cEcE2A41038d', 'tkn0_key': 'WETH-6Cc2', 'tkn1_key': 'LHINU-038d', 'tkn0_decimals': 18.0, 'tkn1_decimals': 18.0, 'exchange_id': 4.0, 'tkn0_symbol': 'WETH', 'tkn1_symbol': 'LHINU', 'timestamp': nan, 'tkn0_balance': nan, 'tkn1_balance': nan, 'liquidity': 0, 'sqrt_price_q96': 377342446735349330393098223164703, 'tick': 169379, 'tick_spacing': 60, 'exchange': 'uniswap_v3'}


In [64]:
print(pools[0])

{'cid': '0x06b295766e941eb449be6a47d11466a50debbacc20638269523dac448fd210d4', 'last_updated': nan, 'last_updated_block': 17656341, 'descr': 'uniswap_v3 WETH-6Cc2/LHINU-038d 3000', 'pair_name': 'WETH-6Cc2/LHINU-038d', 'exchange_name': 'uniswap_v3', 'fee': 3000, 'fee_float': 0.003, 'address': '0xc5BBA4469eD5B55eE21228C9E7261816081C4Bfb', 'anchor': nan, 'tkn0_address': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'tkn1_address': '0xCeDefE438860D2789dA6419b3a19cEcE2A41038d', 'tkn0_key': 'WETH-6Cc2', 'tkn1_key': 'LHINU-038d', 'tkn0_decimals': 18.0, 'tkn1_decimals': 18.0, 'exchange_id': 4.0, 'tkn0_symbol': 'WETH', 'tkn1_symbol': 'LHINU', 'timestamp': nan, 'tkn0_balance': nan, 'tkn1_balance': nan, 'liquidity': 0, 'sqrt_price_q96': 377342446735349330393098223164703, 'tick': 169379, 'tick_spacing': 60, 'exchange': 'uniswap_v3'}


In [65]:
static_pool_data_filename = "static_pool_data"

static_pool_data = pd.read_csv(f"fastlane_bot/data/{static_pool_data_filename}.csv", low_memory=False)
    
uniswap_v2_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v2_event_mappings.csv", low_memory=False)
        
tokens = pd.read_csv("fastlane_bot/data/tokens.csv", low_memory=False)
        
exchanges = "carbon_v1,bancor_v3,uniswap_v3,uniswap_v2,sushiswap_v2"

exchanges = exchanges.split(",")
arb_mode = "bancor_v3"

alchemy_max_block_fetch = 20

In [66]:
static_pool_data["cid"] = [
        cfg.w3.keccak(text=f"{row['descr']}").hex()
        for index, row in static_pool_data.iterrows()
    ]


In [67]:
# Filter out pools that are not in the supported exchanges
static_pool_data = [
    row for index, row in static_pool_data.iterrows()
    if row["exchange_name"] in exchanges
]

static_pool_data = pd.DataFrame(static_pool_data)

In [68]:
static_pool_data['exchange_name'].unique()

array(['uniswap_v3', 'bancor_v3', 'uniswap_v2', 'sushiswap_v2'],
      dtype=object)

In [69]:
from joblib import Parallel, delayed

# Initialize data fetch manager
mgr = Manager(
    web3=cfg.w3,
    cfg=cfg,
    pool_data=static_pool_data.to_dict(orient="records"),
    SUPPORTED_EXCHANGES=exchanges,
    alchemy_max_block_fetch=alchemy_max_block_fetch,
    uniswap_v2_event_mappings=uniswap_v2_event_mappings,
    tokens=tokens.to_dict(orient="records"),
)

# Add initial pools for each row in the static_pool_data
start_time = time.time()
Parallel(n_jobs=-1, backend="threading")(
    delayed(mgr.add_pool_to_exchange)(row) for row in mgr.pool_data
)
cfg.logger.info(f"Time taken to add initial pools: {time.time() - start_time}")

# check if any duplicate cid's exist in the pool data
mgr.deduplicate_pool_data()
cids = [pool["cid"] for pool in mgr.pool_data]
assert len(cids) == len(set(cids)), "duplicate cid's exist in the pool data"

2023-07-09 06:54:14,640 [fastlane:INFO] - Time taken to add initial pools: 0.0918729305267334
2023-07-09 06:54:14,640 [fastlane:INFO] - Time taken to add initial pools: 0.0918729305267334
2023-07-09 06:54:14,640 [fastlane:INFO] - Time taken to add initial pools: 0.0918729305267334


In [70]:
def init_bot(mgr: Manager) -> CarbonBot:
    """
    Initializes the bot.

    Parameters
    ----------
    mgr : Manager
        The manager object.

    Returns
    -------
    CarbonBot
        The bot object.
    """
    mgr.cfg.logger.info("Initializing the bot...")
    bot = CarbonBot(ConfigObj=mgr.cfg)
    bot.db = db
    assert isinstance(
        bot.db, QueryInterface
    ), "QueryInterface not initialized correctly"
    return bot

In [71]:
bot = init_bot(mgr)

2023-07-09 06:54:14,657 [fastlane:INFO] - Initializing the bot...
2023-07-09 06:54:14,657 [fastlane:INFO] - Initializing the bot...
2023-07-09 06:54:14,657 [fastlane:INFO] - Initializing the bot...


In [72]:
# add data cleanup steps from main.py
bot.db.handle_token_key_cleanup()
bot.db.remove_unmapped_uniswap_v2_pools()
bot.db.remove_zero_liquidity_pools()
bot.db.remove_unsupported_exchanges()

2023-07-09 06:54:14,793 [fastlane:INFO] - Removed 3266 unmapped uniswap_v2/sushi pools. 1924 uniswap_v2/sushi pools remaining
2023-07-09 06:54:14,793 [fastlane:INFO] - Removed 3266 unmapped uniswap_v2/sushi pools. 1924 uniswap_v2/sushi pools remaining
2023-07-09 06:54:14,793 [fastlane:INFO] - Removed 3266 unmapped uniswap_v2/sushi pools. 1924 uniswap_v2/sushi pools remaining
2023-07-09 06:54:14,795 [fastlane:INFO] - Unmapped uniswap_v2/sushi pools:
2023-07-09 06:54:14,795 [fastlane:INFO] - Unmapped uniswap_v2/sushi pools:
2023-07-09 06:54:14,795 [fastlane:INFO] - Unmapped uniswap_v2/sushi pools:
2023-07-09 06:54:15,070 [fastlane:INFO] - uniswap_v3: 0
2023-07-09 06:54:15,070 [fastlane:INFO] - uniswap_v3: 0
2023-07-09 06:54:15,070 [fastlane:INFO] - uniswap_v3: 0
2023-07-09 06:54:15,071 [fastlane:INFO] - uniswap_v2: 3266
2023-07-09 06:54:15,071 [fastlane:INFO] - uniswap_v2: 3266
2023-07-09 06:54:15,071 [fastlane:INFO] - uniswap_v2: 3266
2023-07-09 06:54:15,072 [fastlane:INFO] - sushiswap_

In [73]:
tokens = bot.db.get_tokens()
tokens

[Token(symbol='RLB', address='0x046EeE2cc3188071C02BfC1745A6b17c656e3f3d', decimals=18, key='RLB-3f3d'),
 Token(symbol='REQ', address='0x8f8221aFbB33998d8584A2B05749bA73c37a938a', decimals=18.0, key='REQ-938a'),
 Token(symbol='STG', address='0xAf5191B0De278C7286d6C7CC6ab6BB8A73bA2Cd6', decimals=18.0, key='STG-2Cd6'),
 Token(symbol='BEN', address='0x9bf1D7D63dD7a4ce167CF4866388226EEefa702E', decimals=18.0, key='BEN-702E'),
 Token(symbol='HEX', address='0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39', decimals=8, key='HEX-eb39'),
 Token(symbol='TUCKER', address='0x6f49694827d46A14Baf2a7b1eAC3e6eB3526A84f', decimals=9, key='TUCKER-A84f'),
 Token(symbol='MONA', address='0x275f5Ad03be0Fa221B4C6649B8AeE09a42D9412A', decimals=18.0, key='MONA-412A'),
 Token(symbol='ENS', address='0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72', decimals=18.0, key='ENS-9D72'),
 Token(symbol='POND', address='0x57B946008913B82E4dF85f501cbAeD910e58D26C', decimals=18.0, key='POND-D26C'),
 Token(symbol='BIAO', address='0x00

In [74]:
ADDRDEC = {t.key: (t.address, int(t.decimals)) for t in tokens if not math.isnan(t.decimals)}
# ADDRDEC = {t.key: (t.address, int(t.decimals)) for t in tokens}

In [75]:

print(len(ADDRDEC))

99


In [76]:
flashloan_tokens = bot.setup_flashloan_tokens(["BNT-FF1C"])
CCm = bot.setup_CCm(None)

In [77]:
pools = db.get_pool_data_with_tokens()

In [78]:
print(len(pools))

283


In [79]:
# single = bot.run_single_mode(flashloan_tokens, CCm, arb_mode, run_data_validator=True)

single = bot._run(flashloan_tokens=flashloan_tokens, CCm=CCm, arb_mode=arb_mode, data_validator=True, result="calc_trade_instr")


2023-07-09 06:54:15,144 [fastlane:WARNING] - base_exchange must be bancor_v3 for single_triangle_bancor3, setting it to bancor_v3
2023-07-09 06:54:15,144 [fastlane:WARNING] - base_exchange must be bancor_v3 for single_triangle_bancor3, setting it to bancor_v3
2023-07-09 06:54:15,144 [fastlane:WARNING] - base_exchange must be bancor_v3 for single_triangle_bancor3, setting it to bancor_v3
2023-07-09 06:54:15,145 [fastlane:INFO] - flashloan_tokens for arb_mode=single_triangle_bancor3 will be overwritten. 
2023-07-09 06:54:15,145 [fastlane:INFO] - flashloan_tokens for arb_mode=single_triangle_bancor3 will be overwritten. 
2023-07-09 06:54:15,145 [fastlane:INFO] - flashloan_tokens for arb_mode=single_triangle_bancor3 will be overwritten. 
/Users/mikewcasale/Local/projects/bancor/carbonbot/fastlane_bot/tools/optimizer.py:1310: RuntimeWarning: divide by zero encountered in scalar divide
  price = get(p, tokens_ix.get(tknb)) / get(p, tokens_ix.get(tknq))
/Users/mikewcasale/Local/projects/banco

In [80]:
# bot.run(
#                 polling_interval=12,
#                 flashloan_tokens=["BNT"],
#                 mode="single",
#                 arb_mode="bancor_v3",
#                 run_data_validator=True
#             )